In [1]:
import cv2
import numpy as np
from tqdm import trange
from flygym import Fly, Simulation, Camera, get_data_path
from flygym.arena import FlatTerrain
from flygym.preprogrammed import all_leg_dofs
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
import sys
#sys.path.append('C:/Users/laeti/Documents/laetitia/EPFL/Master/CoursMA2/Control_Behaviour/projet')
from hybrid_turning_fly import HybridTurningFly
from simulation_CPG import Simulation_CPG
from flygym.examples.cpg_controller import CPGNetwork
from IPython import display
from flygym.vision import save_video_with_vision_insets

In [2]:
class VisualTaxisFly(HybridTurningFly):
    def __init__(self, obj_threshold=0.15, decision_interval=0.05, **kwargs):
        super().__init__(**kwargs, enable_vision=True)
        self.obj_threshold = obj_threshold
        self.decision_interval = decision_interval
        self.num_substeps = int(self.decision_interval / self.timestep)
        self.visual_inputs_hist = []

        self.coms = np.empty((self.retina.num_ommatidia_per_eye, 2))

        for i in range(self.retina.num_ommatidia_per_eye):
            mask = self.retina.ommatidia_id_map == i + 1
            self.coms[i, :] = np.argwhere(mask).mean(axis=0)

    def process_visual_observation(self, vision_input):
        features = np.zeros((2, 3))

        for i, ommatidia_readings in enumerate(vision_input):
            is_obj = ommatidia_readings.max(axis=1) < self.obj_threshold
            is_obj_coords = self.coms[is_obj]

            if is_obj_coords.shape[0] > 0:
                features[i, :2] = is_obj_coords.mean(axis=0)

            features[i, 2] = is_obj_coords.shape[0]

        features[:, 0] /= self.retina.nrows  # normalize y_center
        features[:, 1] /= self.retina.ncols  # normalize x_center
        features[:, 2] /= self.retina.num_ommatidia_per_eye  # normalize area
        return features.ravel().astype("float32")

    @staticmethod
    def calc_ipsilateral_speed(deviation, is_found):
        if not is_found:
            return 1.0
        else:
            return np.clip(1 - deviation * 3, 0.4, 1.2)

In [3]:
class MovingFly(Fly):
    def __init__(self, init_pose="stretch", actuated_joints=None, control="position",
                 initial_position=None, initial_orientation=None, **kwargs):
        super().__init__(**kwargs, init_pose=init_pose, actuated_joints=actuated_joints, control=control,
                         spawn_pos=initial_position, spawn_orientation=initial_orientation, enable_vision=True)
        self.visual_inputs_hist = []

    def simulate_step(self, sim: Simulation, yaw_angle: float, perform_movement: bool = False, side: str='L'):
        if perform_movement:
            action = {"joints": self.simulate_movement(sim, yaw_angle, side)}
        else:
            action = {"joints": self.obs["joints"][0].copy()} 
        action["adhesion"] = np.array([1,0,1,1,0,1]) #add adhesion 
        return action

    def simulate_movement(self, sim: Simulation, yaw_angle: float, side: str):
        joint_pos = self.obs["joints"][0].copy()

        if side == "L":
            joint_pos[8:14] = 0 #straigten the leg
            joint_pos[7], joint_pos[9] = 1.57, yaw_angle  # Setting specific yaw and pitch
        else:
            joint_pos[29:35] = 0 #straigten the leg
            joint_pos[28], joint_pos[30] = 1.57, yaw_angle

        return joint_pos


In [4]:
def scenario(start_postion=0):
    pos = (0,4.5,0)
    orien_fwd = (0,0,np.pi/2)
    orien_bcd =(0,0,-np.pi/2)

    match start_postion:
        case 0 : return pos, orien_fwd
        case 1:
            pos = (16,4.5,0)
            return pos, orien_bcd
        case 2:
            pos = (16,-4.5,0)
            return pos, orien_bcd
        case 3 :
            return (0,-4.5,0), orien_fwd
        case TypeError:
            print("wrong start_position, default values taken")
            return pos, orien_fwd

In [5]:
timestep = 1e-4
scenario_fly1=0

fly = Fly(
    name="1",
    init_pose="stretch",
    actuated_joints=all_leg_dofs,
    control="position",
    enable_adhesion=True,
    draw_adhesion=False,
    spawn_pos= scenario(scenario_fly1)[0],
    spawn_orientation = scenario(scenario_fly1)[1]
)

#need to process visual informations
# fly0 = VisualTaxisFly(
#     name="0", #static
#     timestep=timestep,
#     enable_adhesion=True,
#     head_stabilization_model="thorax",
#     neck_kp=1000,
#     spawn_pos=(8,0,0), #position
#     spawn_orientation = (0,0,-np.pi/2)
# )

fly0 = MovingFly(

    name="0", #static
    init_pose="stretch",
    actuated_joints=all_leg_dofs,
    control="position",
    enable_adhesion=True,
    head_stabilization_model="thorax",
    neck_kp=1000,
    initial_position=(8,0,0), #position
    initial_orientation = (0,0,-np.pi/2)
)


arena = FlatTerrain()

birdeye_cam_zoom = arena.root_element.worldbody.add(
    "camera",
    name="birdeye_cam_zoom",
    mode="fixed",
    pos=(15, 0, 20),
    euler=(0, 0, 0),
    fovy=45,
)

birdeye_cam = arena.root_element.worldbody.add(
    "camera",
    name="birdeye_cam",
    mode="fixed",
    pos=(15, 0, 35),
    euler=(0, 0, 0),
    fovy=45,
)

cam = Camera(
    fly=fly0,
    camera_id="birdeye_cam",
    play_speed=0.5,
    window_size=(800, 608),
)

sim = Simulation(
    flies=[fly0, fly],
    cameras=[cam],
    arena=arena,
    timestep=timestep,
)

simulation_cpg = Simulation_CPG(timestep)

In [6]:
#init
run_time = 1
obs, info = sim.reset(seed=0)

for i in fly.model.find_all("geom"):
    sim.physics.named.model.geom_rgba[f"1/{i.name}"] = (0, 0, 0, 1)


second_cam_frames = []

#fly0_action = {"joints": np.array([joints_angles[dof] for dof in fly0.actuated_joints])}
fly0_action = {"joints": np.zeros(len(fly0.actuated_joints)), "adhesion": np.array([1,0,1,1,0,1])}

x = None
alpha = 1e-1

In [7]:
#Load the vision model

from utils import crop_hex_to_rect

import torch
from torch import nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # Define your layers here
        self.conv1 = nn.Conv2d(2, 8, 3, groups=2)  # Convolutional layer
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 8, 3, groups=2)  # Convolutional layer
        self.bn2 = nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8, 8, 3, groups=2)  # Convolutional layer
        self.pool3 = nn.MaxPool2d(2, 2)  # Max pooling layer
        self.fc1 = nn.LazyLinear(16)  # Lazy linear layer
        self.fc2 = nn.Linear(16, 16)  # Linear layer
        self.fc3 = nn.Linear(16, 2)   # Linear layer for output

    def forward(self, x):
        # Define the forward pass
        x = F.tanh(self.conv1(x))
        x = F.relu(self.bn1(x))
        x = F.tanh(self.conv2(x))
        x = self.bn2(x)  # Apply pooling after the second convolution
        x = F.tanh(self.conv3(x))
        x = self.pool3(x)  # Apply pooling after the third convolution
        x = x.flatten(1)   # Flatten the output to feed into linear layers
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x
    

# Create an instance of the model
model = Model()

# Load the model weights
model.load_state_dict(torch.load('./best_model/best_model_vision.pth'))

model.eval()


/Users/hugo/miniconda3/envs/flygymv1/lib/python3.11/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Model(
  (conv1): Conv2d(2, 8, kernel_size=(3, 3), stride=(1, 1), groups=2)
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), groups=2)
  (bn2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), groups=2)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): LazyLinear(in_features=0, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)

In [8]:
for _ in trange(int(run_time / sim.timestep)):
    fly1_action = simulation_cpg.update(fly.actuated_joints)
    obs, _,_ ,_ , info = sim.step({
        "0" : fly0_action,
        "1" : fly1_action,
    })

    obs0, info0 = obs["0"], info["0"]
    fly0.obs = obs0
    render_res = sim.render()[0]

    if render_res is not None:
        fly0.visual_inputs_hist.append(obs0["vision"].copy())
        second_cam = sim.physics.bind(birdeye_cam_zoom)

        x_new = sim._get_center_of_mass()[0]

        if x is None:
            x = x_new

        x = (1 - alpha) * x + alpha * x_new

        second_cam.pos[0] = x
        second_img = sim.physics.render(
            width=700, height=560, camera_id="birdeye_cam_zoom"
        )
        second_img = cv2.putText(
            np.ascontiguousarray(second_img),
            f"{sim.cameras[0].play_speed}x",
            org=(20, 30),
            fontFace=cv2.FONT_HERSHEY_DUPLEX,
            fontScale=0.8,
            color=(0, 0, 0),
            lineType=cv2.LINE_AA,
            thickness=1,
        )
        second_cam_frames.append(second_img)

    #visual_features = fly0.process_visual_observation(obs0["vision"])
        

    #process the image :
        
    imgs = crop_hex_to_rect(obs0["vision"])
    imgs = np.expand_dims(imgs, axis=0) #transform into a 4d vector to match the training data

    with torch.no_grad(): #gradient doesn't flow back in the model
        coords_lr_pred = model(torch.tensor(imgs)).numpy()

    #prediction of the other fly's position
    theta_pred = np.angle(coords_lr_pred @ (1, -1j) * np.exp(1j * np.pi / 4))
    distances = np.linalg.norm(coords_lr_pred, axis=1)

    #print(theta_pred,distances)

    #defining side and offset -> front to the side 
    if theta_pred > 0:
        side='R'
        real_angle = theta_pred + np.deg2rad(90) #offset on the leg actuator
    else:
        side='L'
        real_angle = theta_pred - np.deg2rad(90) #offset on the leg actuator

    #getting the action based on the observer 
    if distances<=2: #to change when it's done
        fly0_action = fly0.simulate_step(sim, yaw_angle=real_angle, perform_movement=True, side=side)
    else:
        fly0_action = fly0.simulate_step(sim, yaw_angle=real_angle, perform_movement=False, side=side)
    
cam.save_video("./outputs/cpg_controller_with_adhesion.mp4", 0)

 25%|██▌       | 2508/10000 [00:17<01:01, 121.93it/s]/var/folders/9b/zp5p8_495653n52_yn4hbq9h0000gn/T/ipykernel_2072/1779317081.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  joint_pos[7], joint_pos[9] = 1.57, yaw_angle  # Setting specific yaw and pitch
 26%|██▌       | 2595/10000 [00:18<00:46, 159.63it/s]/var/folders/9b/zp5p8_495653n52_yn4hbq9h0000gn/T/ipykernel_2072/1779317081.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  joint_pos[28], joint_pos[30] = 1.57, yaw_angle
 26%|██▋       | 2639/10000 [00:18<00:51, 142.92it/s]


PhysicsError: Physics state is invalid. Warning(s) raised: mjWARN_BADQACC

In [ ]:
display.Video("./outputs/cpg_controller_with_adhesion.mp4")

In [ ]:
birdeye_cam_frames = cam._frames
cam._frames = second_cam_frames

In [ ]:
sim.fly = fly0

save_video_with_vision_insets(
    sim,
    cam,
    "outputs/fly_following_with_retina_images.mp4",
    fly0.visual_inputs_hist,
)

In [ ]:
display.Video("./outputs/fly_following_with_retina_images.mp4")